mlexperience.org - news article classification
 
http://www.nltk.org/book/ch02.html help is available in the nltk book section 1.4
http://enricbaltasar.com/python-summary-methods-lists/ help with Python lists
http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
Pandas dataframe guide

Investigating word popularity in the articles

In [1]:
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import StanfordTokenizer
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
from nltk.corpus import reuters
import os
from nltk.parse import stanford
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import RegexpStemmer
st = RegexpStemmer('ing$|s$|e$|able$', min=4)
from nltk.stem.snowball import EnglishStemmer
snow = EnglishStemmer()
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from collections import Counter
import numpy as np
import pandas as pd
#os.environ['STANFORD_PARSER'] = '/Users/David/postagger/stanford-parser-full-2014-08-27/stanford-parser.jar'
#os.environ['STANFORD_MODELS'] = '/Users/David/postagger/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models.jar'
#parser = stanford.StanfordParser(model_path="/Users/David/postagger/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models/edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
cachedStopWords = stopwords.words("english")
#cachedStopWords.append('said')
 
def tokenize(text):
    min_length = 3
    #tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    words =  map(lambda word: word.lower(), WordPunctTokenizer().tokenize(text))
    #words = list(WhitespaceTokenizer().span_tokenize(text))
    #words = map(lambda word: word.lower(), word_tokenize(text));    
    #words = wordpunct_tokenize(text)
    #word_tokenize = Penn Treebank#
    #words = map(lambda word: word.lower(), tokenizer.tokenize(text));
    #words = list(parser.raw_parse(text,verbose=False))
    #words = map(lambda word: word.lower(), text.split())
    words = [word for word in words
             if word not in cachedStopWords]
    #tokens =(list(map(lambda token: PorterStemmer().stem(token),words)));
    #tokens =(list(map(lambda token: LancasterStemmer().stem(token),words)));
    #tokens =(list(map(lambda token: snow.stem(token),words)));
    #tokens =(list(map(lambda token: st.stem(token),words)));
    tokens =(list(map(lambda token: wnl.lemmatize(token),words)));
    #tokens=words
    p = re.compile('[a-zA-Z]+');
    filter_words = list(filter(lambda token: p.match(token) and len(token)>=min_length, tokens));
    return filter_words


In [2]:
import pandas as pd  #For python data frame
import pymongo #For mongoDB
#important
#run guardian.py first to create the data set
#guardian.py 
#start_date = date.today() - timedelta(2)  -  2 days from the current day
#end_date = date.today() - timedelta(1) - end 1 day from the current day
#to get a longer period change the timedelta(x) 


#Create connection to the MongoDB. Note the MongoDB must be running.
conn=pymongo.MongoClient('localhost',27017)
db = conn.mlexperience  #This database needs to be changed according to date of scrape
#Robomonogo 0.9.0 is useful to see the database and json structures of documents
#https://robomongo.org/

print(db.news.count())  #Print out the number of records in the database, visual check on connection.

df = pd.DataFrame(list(db.news.find())) #Copy the reports to the dataframe and close the connection
conn.close() 

247


In [4]:
#build a data frame to feed the vectorizer
data1 = pd.DataFrame(data=df)
df = data1[['ml_id','topic','text']]


In [5]:
df.shape


(247, 3)

In [6]:
df.tail(5)

,ml_id,topic,text
242,world/2017/may/08/pepe-the-frog-creator-kills-...,/World news,The creator of Pepe the Frog has symbolically ...
243,artanddesign/2017/may/08/bare-naked-photos-hig...,/Art and design,It seems like an awkward conundrum for a photo...
244,tv-and-radio/2017/may/08/american-gods-recap-s...,/Television & radio,Spoiler alert: this blog is for people watchin...
245,australia-news/2017/may/08/labor-australians-f...,/Australia news,Bill Shorten has conceded that people with mor...
246,commentisfree/2017/may/08/naplan-is-a-mess-and...,/Opinion,There was no Naplan when I started teaching in...


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#Values for document frequencies in percent.
minp=0.001
maxp=0.95
vec6_count = CountVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')
vec6_idf =   TfidfVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=10)
#hv.transform(corpus)
vec=vec6_count

#2# Specify the model
from sklearn.naive_bayes import MultinomialNB
#clf = MultinomialNB()
from sklearn import neighbors
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn import svm
#clf = svm.SVC()
#n_neighbors = 15
#clf = neighbors.KNeighborsClassifier(n_neighbors)
#clf = neighbors.RadiusNeighborsClassifier(n_neighbors)
from sklearn.neural_network import MLPClassifier
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
#3# Using pipelineing, the data transformations can chained together 
#and the last object is the estimator or model.

from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2")
from sklearn import pipeline
pipe = pipeline.Pipeline([('vectorizer', vec),
                          ('clf', clf)])

In [9]:
X=df['text']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['topic'])
y=le.transform(df['topic'])

In [10]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.4,random_state=79)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
#6# Train the model using 5 fold cross validation and calculate the scores. 
train_scores = cross_validation.cross_val_score(pipe, X_train, y_train, cv=5)
#Fit the model
pipe=pipe.fit(X_train, y_train)
#Score the model on unseen test data  data
test_score=pipe.score(X_test,y_test)

#7# Print out the accuracy scores and 
print("Training Accuracy: %0.3f (+/- %0.3f). Test Score: (%0.3f)" 
      % (train_scores.mean(), train_scores.std(),test_score))




/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Training Accuracy: 0.413 (+/- 0.076). Test Score: (0.283)


In [12]:
y_predicted = pipe.predict(df['text'])
pred_labels = le.inverse_transform(y_predicted)
act_labels = le.inverse_transform(y_test)

In [13]:
df['predicted'] = pred_labels

/home/david/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
df['score'] = (df['predicted'] == df['topic'])
df['hash'] = 1

/home/david/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/david/.local/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [23]:
temdf = df.drop(['text'], axis=1, level=None, inplace=False, errors='raise')
temdf.head(5)
temdf = temdf[temdf['ml_id'] != 'x']
#temdf['topic'] = temdf['topic'].apply(lambda x: '' + str(x[0]))

In [27]:
table = pd.pivot_table(temdf, values='hash', index='topic', columns='score', aggfunc=np.sum, margins=True, fill_value=0)
table = table.sort('All', ascending=False)
table.head(10)

/home/david/.local/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


score,False,True,All
topic,,,
All,71.0,176.0,247.0
/Opinion,10.0,16.0,26.0
/World news,6.0,19.0,25.0
/Football,1.0,17.0,18.0
/Politics,3.0,12.0,15.0
/Life and style,6.0,7.0,13.0
/US news,2.0,9.0,11.0
/Books,0.0,11.0,11.0
/Environment,2.0,8.0,10.0


In [28]:
table.iloc[:,1] == table.iloc[:,2]
print(table[table.iloc[:,1] == table.iloc[:,2]])

score                              False  True   All
topic                                               
/Books                               0.0  11.0  11.0
/Society                             0.0   6.0   6.0
/Science                             0.0   5.0   5.0
/Film                                0.0   5.0   5.0
/Barclays Let's go forward           0.0   3.0   3.0
/Cities                              0.0   2.0   2.0
/Fashion                             0.0   1.0   1.0
/Membership                          0.0   1.0   1.0
/Healthcare Professionals Network    0.0   1.0   1.0
/News                                0.0   1.0   1.0
/Crosswords                          0.0   1.0   1.0


In [29]:
print(table[table.iloc[:,0] == table.iloc[:,2]])

score                             False  True  All
topic                                             
/Media                              4.0   0.0  4.0
/Art and design                     3.0   0.0  3.0
/Culture                            2.0   0.0  2.0
/Global                             2.0   0.0  2.0
/Discover Canal & River Trust       1.0   0.0  1.0
/Travel                             1.0   0.0  1.0
/Guardian Careers                   1.0   0.0  1.0
/Guardian Small Business Network    1.0   0.0  1.0
/Social Care Network                1.0   0.0  1.0
/Public Leaders Network             1.0   0.0  1.0
/Inequality                         1.0   0.0  1.0


In [30]:
table.shape

(42, 3)

In [31]:
temdf.head(10)

,ml_id,topic,predicted,score,hash
0,world/2017/may/09/philippines-senator-tells-un...,/World news,/World news,True,1
1,football/2017/may/08/ben-gibson-middlesbrough-...,/Football,/Football,True,1
2,sport/2017/may/09/lou-richards-death-finally-k...,/Sport,/Sport,True,1
3,football/2017/may/08/antonio-conte-chelsea-awa...,/Football,/Football,True,1
4,us-news/2017/may/08/sally-yates-trump-russia-m...,/US news,/US news,True,1
5,politics/2017/may/08/corbyn-pins-labours-elect...,/Politics,/Politics,True,1
6,world/2017/may/09/gare-du-nord-evacuated-in-pa...,/World news,/World news,True,1
7,environment/2017/may/09/bq-to-stop-selling-pla...,/Environment,/Books,False,1
8,society/2017/may/09/statin-trial-raises-hope-d...,/Society,/Society,True,1
9,money/2017/may/08/theresa-may-to-promise-price...,/Money,/Money,True,1


In [32]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import neighbors
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn import svm
n_neighbors = 15
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2")
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = MultinomialNB()
clf5 = svm.SVC()
clf6 = neighbors.KNeighborsClassifier(n_neighbors)
clf7 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('mnb', clf4), ('svm', clf5),('knn', clf6), 
                                    ('mlp', clf7), ('sgd', clf)], voting='hard')
vec6_count = CountVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')
vec6_idf =   TfidfVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')

for vec, vec_lab in zip([vec6_count, vec6_idf],['Count vec','tfidf vec']):
    for clf, label in zip([clf, clf1, clf2, clf4, clf5, clf6, clf7, eclf], 
                          ['Stocastic Gradient','Logistic Regression', 'Random Forest', 'naive Bayes', 
                           'svm','knn','nn mlp', 'ensemble']):
        pipe = pipeline.Pipeline([('vectorizer', vec),('clf', clf)])
        train_scores = cross_validation.cross_val_score(pipe, X_train, y_train, cv=5)
        pipe=pipe.fit(X_train, y_train)
        test_score=pipe.score(X_test,y_test)
        print("Vec:  [%s] -> Accuracy: %0.2f (+/- %0.2f) Test: %0.2f  Model: [%s]" 
              % (vec_lab, train_scores.mean(), train_scores.std(), test_score, label))

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Vec:  [Count vec] -> Accuracy: 0.39 (+/- 0.04) Test: 0.21  Model: [Stocastic Gradient]


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Vec:  [Count vec] -> Accuracy: 0.49 (+/- 0.09) Test: 0.33  Model: [Logistic Regression]


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


...

Vec:  [tfidf vec] -> Accuracy: 0.40 (+/- 0.08) Test: 0.31  Model: [knn]


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Vec:  [tfidf vec] -> Accuracy: 0.11 (+/- 0.02) Test: 0.10  Model: [nn mlp]


/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Vec:  [tfidf vec] -> Accuracy: 0.29 (+/- 0.06) Test: 0.20  Model: [ensemble]
